# Clusterização

Os algoritmos de clusterização ou agrupamento permitem que seja feito agrupamentos com bases nas semelhanças encontradas. É uma técnica que permite realizar a divisão de grupos em um conjunto de dados de forma automática baseado em medidas de similaridades ou de distância. Existem vários métodos que permitem obter medidas de similaridade, podemos citar a similaridade de cosseno e a correlação de pearson

### São utilizados para:

* Segmentação de mercado
* Agrupamento de características de clientes, produtos mais similares
* Agrupar perfis de clientes (Amazon Prime, Netflix, similares)
* Recomendação de itens

### Definição
* A clusterisação, que é uma técnica de aprendizado não supervisionado que permite dividir automaticamente o conjunto de dados em grupos de acordo com uma similaridade

## Implementando algoritmo de K-means

Nesta atividade, será implementado o algoritmo de K-means em um conjunto de dados. Para isso, vamos executar os seguintes passos

* Carregar o dataset de iris;
* Análisar os dados coletados;
* Realizar o tratamento de dados;
* Aplicar modelo de ML de K-means
* Visualizar os resultados obtidos;


## Importando bibliotecas

In [41]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans

### Verificando versões das bibliotecas

In [42]:
import sklearn, plotly

print(f'''Verificando as versões das bibliotecas\n
pandas: {pd.__version__}\n
numpy: {np.__version__}\n
sklearn: {sklearn.__version__}\n
plotly: {plotly.__version__}''')

Verificando as versões das bibliotecas

pandas: 1.5.3

numpy: 1.22.4

sklearn: 1.2.2

plotly: 5.13.1


## Coleta de dados

In [43]:
iris = pd.read_csv('/content/iris.csv', sep=',')

## Análise dos dados coletados

In [44]:
# Visualização do arquivo
iris.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [45]:
# Visualização do arquivo
iris.tail()

,sepal.length,sepal.width,petal.length,petal.width,variety
145,6.7,3.0,5.2,2.3,Virginica
146,6.3,2.5,5.0,1.9,Virginica
147,6.5,3.0,5.2,2.0,Virginica
148,6.2,3.4,5.4,2.3,Virginica
149,5.9,3.0,5.1,1.8,Virginica


In [46]:
# Gerando algumas estatisticas da base de dados
iris.describe()

,sepal.length,sepal.width,petal.length,petal.width
count,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333
std,0.828066,0.435866,1.765298,0.762238
min,4.300000,2.000000,1.000000,0.100000
25%,5.100000,2.800000,1.600000,0.300000
50%,5.800000,3.000000,4.350000,1.300000
75%,6.400000,3.300000,5.100000,1.800000
max,7.900000,4.400000,6.900000,2.500000


In [47]:
# Número de classes existentes no dataframe
iris['variety'].unique()

array(['Setosa', 'Versicolor', 'Virginica'], dtype=object)

In [48]:
# Contando numero de classes
len(iris['variety'].unique())

3

In [49]:
# Avaliando número de linhas e colunas do dataset
iris.shape

(150, 5)

In [50]:
# Verificando se existe algum valor nulo ou ausente no dataset
iris.isna().sum()

sepal.length    0
sepal.width     0
petal.length    0
petal.width     0
variety         0
dtype: int64

In [51]:
# Verificando informações da base de dados
iris.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   sepal.length  150 non-null    float64
 1   sepal.width   150 non-null    float64
 2   petal.length  150 non-null    float64
 3   petal.width   150 non-null    float64
 4   variety       150 non-null    object 
dtypes: float64(4), object(1)
memory usage: 6.0+ KB


## Criando clusters com os dados das pétalas
___
Realizando o agrupamento dos dados com base no comprimento e largura das pétalas 

### Coletando e Normalizando dados das pétalas

In [52]:
iris.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [53]:
# Criando variável X com as colunas petal.length	petal.width
x_petalas = iris.loc[:,['petal.length',	'petal.width']].values
x_petalas[:10]

array([[1.4, 0.2],
       [1.4, 0.2],
       [1.3, 0.2],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.7, 0.4],
       [1.4, 0.3],
       [1.5, 0.2],
       [1.4, 0.2],
       [1.5, 0.1]])

#### Normalizando os dados
Quando os dados estão em uma escala muito diferente, precisamos normalizar os dados colocando no mesmo padrão e na mesma escola.
Como o Kmeans realiza os cálculos baseado na distância, devamos padronizar os dados afim de que um atributo não seja considerado mais importante do que o outro.

**OBS:** Não seria necessária esta etapa para este dataset, devido a baixa diferenciação dos valores minimos e máximos.
Esta etapa será apenas para fins de aprendizado.

In [54]:
normalizar = StandardScaler()

# Normalização dos dados
x_petalas = normalizar.fit_transform(x_petalas)
x_petalas[:10]

array([[-1.34022653, -1.3154443 ],
       [-1.34022653, -1.3154443 ],
       [-1.39706395, -1.3154443 ],
       [-1.2833891 , -1.3154443 ],
       [-1.34022653, -1.3154443 ],
       [-1.16971425, -1.05217993],
       [-1.34022653, -1.18381211],
       [-1.2833891 , -1.3154443 ],
       [-1.34022653, -1.3154443 ],
       [-1.2833891 , -1.44707648]])

### Calculando valor do WCSS
Para calcular o número de clusters será utilizado o método do cotovelo WCSS é a soma da distância quadrada entre cada ponto e o centróide em um cluster

In [55]:
wcss_petala = []
for i in range(1, 11):
  kmeans_petala = KMeans(n_clusters=i, random_state=0)
  kmeans_petala.fit(x_petalas)
  wcss_petala.append(kmeans_petala.inertia_)

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:8

In [56]:
# Visualizando os valores de wcss
for i in range(len(wcss_petala)):
  print(f"Cluster: {i+1} - Valor do WCSS: {wcss_petala[i]}")

Cluster: 1 - Valor do WCSS: 300.0
Cluster: 2 - Valor do WCSS: 54.16878133149558
Cluster: 3 - Valor do WCSS: 18.02696261254407
Cluster: 4 - Valor do WCSS: 12.283372197379123
Cluster: 5 - Valor do WCSS: 9.152134379409805
Cluster: 6 - Valor do WCSS: 7.187603589633753
Cluster: 7 - Valor do WCSS: 5.994729609966573
Cluster: 8 - Valor do WCSS: 5.144963679454987
Cluster: 9 - Valor do WCSS: 4.402436506349908
Cluster: 10 - Valor do WCSS: 3.8964398329646883


### Definindo melhor número de clusters para as pétalas

In [57]:
graph_cotovelo_petala = px.line(x=range(1, 11), y=wcss_petala)
graph_cotovelo_petala.update_xaxes(title_text='Número de Clusters')
graph_cotovelo_petala.update_yaxes(title_text='Valor de WCSS')
graph_cotovelo_petala.update_layout(title_text='Definindo o número de Clusters', title_x=0.5)
graph_cotovelo_petala.show()

## Executando o algoritmo de K-means para as pétalas


In [58]:
# Número de clusters = 3
kmeans_petala = KMeans(n_clusters=3, random_state=0)
label_cluster_petala = kmeans_petala.fit_predict(x_petalas)

# Verificando classificação dos clusters
label_cluster_petala

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int32)

## Criando clusters com os dados das sépalas
___
Realização do agrupamento dos dados baseado no comprimento e na largura das sépalas.

### Coletando e Normalizando os dados das sépalas

In [59]:
# Realizando agrupamento das sepalas
x_sepala = iris.loc[:, ['sepal.length',	'sepal.width']].values

# Normalização de dados
x_sepala = normalizar.fit_transform(x_sepala)
x_sepala[:5]

array([[-0.90068117,  1.01900435],
       [-1.14301691, -0.13197948],
       [-1.38535265,  0.32841405],
       [-1.50652052,  0.09821729],
       [-1.02184904,  1.24920112]])

### Calculando valor do WCSS

In [60]:
wcss_sepala = []
for i in range(1, 11):
  kmeans_sepala = KMeans(n_clusters=i, random_state=0, init='k-means++', n_init=10)
  kmeans_sepala.fit(x_sepala)
  wcss_sepala.append(kmeans_sepala.inertia_)

In [61]:
# Visualizando os valores de wcss
for i in range(len(wcss_sepala)):
  print(f"Cluster: {i+1} - Valor do WCSS: {wcss_sepala[i]}")

Cluster: 1 - Valor do WCSS: 300.0
Cluster: 2 - Valor do WCSS: 166.9517093393448
Cluster: 3 - Valor do WCSS: 102.61686896314461
Cluster: 4 - Valor do WCSS: 79.76108196298645
Cluster: 5 - Valor do WCSS: 61.80924909521247
Cluster: 6 - Valor do WCSS: 52.53666909138647
Cluster: 7 - Valor do WCSS: 44.416621911377995
Cluster: 8 - Valor do WCSS: 36.18434119518747
Cluster: 9 - Valor do WCSS: 30.51194838055993
Cluster: 10 - Valor do WCSS: 26.667787013897346


### Definindo melhor número de clusters para as sépalas

In [62]:
# Visualizando os valores de wcss
graph_cotovelo_sepala = px.line(x=range(1, 11), y=wcss_sepala)
graph_cotovelo_sepala.update_xaxes(title_text='Número de Clusters')
graph_cotovelo_sepala.update_yaxes(title_text='Valor de WCSS')
graph_cotovelo_sepala.update_layout(title_text='Definindo o número de Clusters', title_x=0.5)
graph_cotovelo_sepala.show()

## Executando o ALgoritmo K-means para as sépalas

In [63]:
# Definir melhor número de clusters = 3
kmeans_sepala = KMeans(n_clusters=3, random_state=0)
label_cluster_sepalas = kmeans_sepala.fit_predict(x_sepala)

# Verificando classificação dos clusters
label_cluster_sepalas

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning:

The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning



array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1, 2,
       1, 1, 1, 1, 2, 1, 1, 1, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 1, 2, 1, 2,
       2, 1, 2, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 2, 2,
       2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1], dtype=int32)

## Visualização dos dados

### Criando uma função para mostrar os gráficos

In [64]:
def gerar_grafico(X, Y, color, X_centroides, y_centroides):
  graph = px.scatter(x = X, y = Y, color = color)
  graph_centroide = px.scatter(x = X_centroides, y = y_centroides, size=[5, 5, 5])
  graph = go.Figure(data=graph.data + graph_centroide.data)
  graph.update_xaxes(title_text='Comprimento')
  graph.update_yaxes(title_text='Largura')
  graph.update_layout(title_text='Clusters', title_x=0.5)
  graph.show()

### Executando a função para as sépalas

In [65]:
# Visualizando centroides
centroides_sepala = kmeans_sepala.cluster_centers_
centroides_sepala

array([[-1.00206653,  0.90625492],
       [-0.10143926, -0.94652188],
       [ 1.10971635,  0.09821729]])

In [66]:
# Visualizando dados agrupados para as sépalas
X = x_sepala[:,0]
y = x_sepala[:,1]
color = label_cluster_sepalas
X_centroides = centroides_sepala[:,0]
y_centroides = centroides_sepala[:,1]

gerar_grafico(X, y, color, X_centroides, y_centroides)

### Executando a função para as pétalas

In [67]:
# Visualizando centroides
centroides_petala = kmeans_petala.cluster_centers_
centroides_petala

array([[-1.30498732, -1.25489349],
       [ 1.02799959,  1.12797813],
       [ 0.3058728 ,  0.16541778]])

In [68]:
# Visualizando dados agrupados com os dados das petalas
X = x_petalas[:,0]
y = x_petalas[:,1]
color = label_cluster_petala
X_centroides = centroides_petala[:,0]
y_centroides = centroides_petala[:,1]

gerar_grafico(X, y, color, X_centroides, y_centroides)